In [2]:
import stokespy
from stokespy import get_HMI_data, get_SP_data, StokesParamCube, StokesParamMap, StokesProfile
from stokespy.instload import get_HMI_data_test
import datetime
import astropy
import astropy.units as u
from astropy.coordinates import SkyCoord, SpectralCoord
from astropy.wcs.wcsapi import SlicedLowLevelWCS, HighLevelWCSWrapper

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import ndcube

matplotlib.use('TkAgg')

# Setup the autoreload function.
%load_ext autoreload
%autoreload 2

# Website example use of project_to method

In [3]:
# Define data array.
data = np.random.rand(4, 4, 5)

# Define WCS transformations in an astropy WCS object.
wcs = astropy.wcs.WCS(naxis=3)
wcs.wcs.ctype = 'WAVE', 'HPLT-TAN', 'HPLN-TAN'
wcs.wcs.cunit = 'Angstrom', 'deg', 'deg'
wcs.wcs.cdelt = 0.2, 0.5, 0.4
wcs.wcs.crpix = 0, 2, 2
wcs.wcs.crval = 10, 0.5, 1

# Instantiate NDCube with supporting data.
my_cube = ndcube.NDCube(data, wcs=wcs)

In [4]:
my_cube.data.shape

(4, 4, 5)

In [8]:
my_cube.wcs

WCS Keywords

Number of WCS axes: 3
CTYPE : 'WAVE'  'HPLT-TAN'  'HPLN-TAN'  
CRVAL : 1e-09  0.5  1.0  
CRPIX : 0.0  2.0  2.0  
PC1_1 PC1_2 PC1_3  : 1.0  0.0  0.0  
PC2_1 PC2_2 PC2_3  : 0.0  1.0  0.0  
PC3_1 PC3_2 PC3_3  : 0.0  0.0  1.0  
CDELT : 2.0000000000000002e-11  0.5  0.4  
NAXIS : 0  0

In [9]:
target_wcs_x = my_cube.wcs

In [10]:
target_wcs_x.wcs.cdelt = [0]

array([2.e-11, 5.e-01, 4.e-01])

In [5]:
# Create a target WCS object with new transformations.
target_wcs = astropy.wcs.WCS(naxis=3)
target_wcs.wcs.ctype = 'WAVE', 'HPLT-TAN', 'HPLN-TAN'
target_wcs.wcs.cunit = 'Angstrom', 'deg', 'deg'
target_wcs.wcs.cdelt = 0.1, 0.5, 0.4
target_wcs.wcs.crpix = 0, 2, 2
target_wcs.wcs.crval = 10, 0.5, 1

reprojected_cube = my_cube.reproject_to(target_wcs=target_wcs, shape_out=(4, 4, 10))

In [11]:
reprojected_cube.wcs

WCS Keywords

Number of WCS axes: 3
CTYPE : 'WAVE'  'HPLT-TAN'  'HPLN-TAN'  
CRVAL : 1e-09  0.5  1.0  
CRPIX : 0.0  2.0  2.0  
PC1_1 PC1_2 PC1_3  : 1.0  0.0  0.0  
PC2_1 PC2_2 PC2_3  : 0.0  1.0  0.0  
PC3_1 PC3_2 PC3_3  : 0.0  0.0  1.0  
CDELT : 1.0000000000000001e-11  0.5  0.4  
NAXIS : 0  0

In [12]:
crop_cube = my_cube[:,:,0:2]

In [16]:
crop_cube.wcs

SlicedLowLevelWCS Transformation

This transformation has 3 pixel and 3 world dimensions

Array shape (Numpy order): None

Pixel Dim  Axis Name  Data size  Bounds
        0  None            None  None
        1  None            None  None
        2  None            None  None

World Dim  Axis Name  Physical Type                   Units
        0  None       em.wl                           m
        1  None       custom:pos.helioprojective.lat  deg
        2  None       custom:pos.helioprojective.lon  deg

Correlation between pixel and world axes:

             Pixel Dim
World Dim    0    1    2
        0  yes   no   no
        1   no  yes  yes
        2   no  yes  yes

In [18]:
my_cube,crop_cube

(<ndcube.ndcube.NDCube object at 0x7fdf67123160>
 NDCube
 ------
 Dimensions: [4. 4. 5.] pix
 Physical Types of Axes: [('custom:pos.helioprojective.lat', 'custom:pos.helioprojective.lon'), ('custom:pos.helioprojective.lat', 'custom:pos.helioprojective.lon'), ('em.wl',)]
 Unit: None
 Data Type: float64,
 NDCube
 ------
 Dimensions: [4. 4. 2.] pix
 Physical Types of Axes: [('custom:pos.helioprojective.lat', 'custom:pos.helioprojective.lon'), ('custom:pos.helioprojective.lat', 'custom:pos.helioprojective.lon'), ('em.wl',)]
 Unit: None
 Data Type: float64)

In [ ]:
%maptlotlib widget

ax1 = plt.subplot(121, projection=my_cube[0,:,:].wcs)
ax2 = plt.subplot(122, projection=crop_cube[0,:,:].wcs)

plt.subplots_adjust(wspace=0.4)

ax1.imshow(lvl2_c_SP.data[0,:,:], origin='lower')
ax2.imshow(cropped_map.data[0,:,:], origin='lower')

#lvl.plot(ax=ax1)
#cropped_map.plot(ax=ax2)

# Example using HMI data

## Load the HMI data

In [2]:
# Select a date and time to search for observations.
#user_date = astropy.time.Time(datetime.datetime(2016, 7, 28, 23, 57, 0), scale='tai')  # Original
#user_date = astropy.time.Time(datetime.datetime(2017, 8, 22, 17, 11, 4), scale='tai')  # AR near limb
user_date = astropy.time.Time(datetime.datetime(2017, 9, 5, 3, 4, 4), scale='tai')  # AR near disc center

# Find the nearest set of Stokes and corresponding inversion results.
#all_fnames_stokes, all_fnames_magvec = get_HMI_data(user_date, user_notify='gdima@hawaii.edu', download=False)
lvl1_data, lvl1_wcs, lvl2_data, lvl2_wcs = \
    get_HMI_data_test(user_date, user_notify='gdima@hawaii.edu', download=False, show_files=False)

Time window used for the search:  <sunpy.net.attrs.Time(2017-09-05 03:04:03.000, 2017-09-05 03:04:05.000)>
User directory pointing to SDO data is not included.
Setting the default directory to: /home/gabriel/Desktop/Science/StokesPY/stokespy_notebooks/Data/SDO/
No download requested. Found 24 Stokes files with unique timestamp(s):
['20170905_030000_TAI']
No download requested. Found 25 inversion files with timestamps: 
['20170905_030000_TAI']
Created data cube with dimensions: (4, 6, 4096, 4096)


Created data cube with dimensions: (1, 4096, 4096)
Filenames used: 


In [3]:
# Create the HMI StokesCubes
lvl1_c_HMI = stokespy.StokesCube(lvl1_data, lvl1_wcs)
print('HMI lvl1 data size: ', lvl1_c_HMI.data.shape)

if lvl2_data.shape[0] == 3:
    lvl2_c_HMI = stokespy.MagVectorCube(lvl2_data, lvl2_wcs)
    print('HMI lvl2 data size: ', lvl2_c_HMI.data.shape)
    

HMI lvl1 data size:  (4, 6, 4096, 4096)


In [4]:
# Create HMI NDcubes
lvl1_NDc_HMI = ndcube.NDCube(lvl1_data, wcs=lvl1_wcs)
lvl2_NDc_HMI = ndcube.NDCube(lvl2_data, wcs=lvl2_wcs)

In [12]:
# Crop to a smaller cube size.
lvl1_NDc_HMI_crop = lvl1_NDc_HMI[:,:,1500:2500, 1500:2500]
print(lvl1_NDc_HMI_crop.data.shape)

(4, 6, 1000, 1000)


In [15]:
# Plot the original.
%matplotlib widget

fig, ax = plt.subplots(subplot_kw={'projection':lvl2_NDc_HMI[0,:,:].wcs})

ax.imshow(lvl2_NDc_HMI.data[0,:,:], origin='lower', cmap='Greys')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Plot the cropped image
%matplotlib widget

fig, ax = plt.subplots(subplot_kw={'projection':lvl1_NDc_HMI_crop[0,0,:,:].wcs})

ax.imshow(lvl1_NDc_HMI_crop.data[0,0,:,:], origin='lower', cmap='Greys')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Reproject to a new WCS

In [17]:
def rotate_cube(orig_cube, theta):
    """ Rotate and reproject cube by some angle theta"""
    if type(theta) is not u.quantity.Quantity:
        theta = theta * u.deg
    print(theta)
    rot_mat = np.asarray([[np.cos(theta), -np.sin(theta)],\
                     [np.sin(theta), np.cos(theta)]])
    
    print(rot_mat)
    
    # Create a copy of the WCS object to modify
    target_wcs_x = astropy.wcs.WCS(orig_cube.wcs.to_header())
    # Apply the rotation.
    target_wcs_x.wcs.pc[0:2,0:2] = rot_mat
    print(target_wcs_x)
    
    return orig_cube.reproject_to(target_wcs=target_wcs_x, 
                                    shape_out=orig_cube.data.shape,)

In [9]:
lvl2_NDc_HMI.data.shape

(1, 4096, 4096)

In [7]:
# Reproject the cube to 0 rotation.
rep_cube_0 = rotate_cube(lvl2_NDc_HMI, 0*u.deg)

0.0 deg
[[ 1. -0.]
 [ 0.  1.]]
WCS Keywords

Number of WCS axes: 3
CTYPE : 'HPLN-TAN'  'HPLT-TAN'  'Parameter'  
CRVAL : 0.0  0.0  0.0  
CRPIX : 2040.244140625  2050.626953125  0.0  
PC1_1 PC1_2 PC1_3  : 1.0  -0.0  0.0  
PC2_1 PC2_2 PC2_3  : 0.0  1.0  0.0  
PC3_1 PC3_2 PC3_3  : 0.0  0.0  1.0  
CDELT : 0.00014009532001283  0.00014009532001283  1.0  
NAXIS : 0  0


In [16]:
# Reproject the cube to 0 rotation.
rep_cube_01 = rotate_cube(lvl2_NDc_HMI, 0*u.deg)

0.0 deg
[[ 1. -0.]
 [ 0.  1.]]
WCS Keywords

Number of WCS axes: 3
CTYPE : 'HPLN-TAN'  'HPLT-TAN'  'Parameter'  
CRVAL : 0.0  0.0  0.0  
CRPIX : 2040.244140625  2050.626953125  0.0  
PC1_1 PC1_2 PC1_3  : 1.0  -0.0  0.0  
PC2_1 PC2_2 PC2_3  : 0.0  1.0  0.0  
PC3_1 PC3_2 PC3_3  : 0.0  0.0  1.0  
CDELT : 0.00014009532001283  0.00014009532001283  1.0  
NAXIS : 0  0


TypeError: reproject_to() got an unexpected keyword argument 'reproject_interp_kw'

In [13]:
import reproject

In [14]:
reproject.__version__

'0.8'

In [11]:
# Reproject the cube to 20 deg rotation
rep_cube_1 = rotate_cube(lvl2_NDc_HMI, 20*u.deg)

20.0 deg
[[ 0.93969262 -0.34202014]
 [ 0.34202014  0.93969262]]
WCS Keywords

Number of WCS axes: 3
CTYPE : 'HPLN-TAN'  'HPLT-TAN'  'Parameter'  
CRVAL : 0.0  0.0  0.0  
CRPIX : 2040.244140625  2050.626953125  0.0  
PC1_1 PC1_2 PC1_3  : 0.9396926207859084  -0.3420201433256687  0.0  
PC2_1 PC2_2 PC2_3  : 0.3420201433256687  0.9396926207859084  0.0  
PC3_1 PC3_2 PC3_3  : 0.0  0.0  1.0  
CDELT : 0.00014009532001283  0.00014009532001283  1.0  
NAXIS : 0  0


In [12]:
# Reproject the cube to 20 deg rotation
rep_cube_2 = rotate_cube(lvl2_NDc_HMI, 45*u.deg)

45.0 deg
[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]
WCS Keywords

Number of WCS axes: 3
CTYPE : 'HPLN-TAN'  'HPLT-TAN'  'Parameter'  
CRVAL : 0.0  0.0  0.0  
CRPIX : 2040.244140625  2050.626953125  0.0  
PC1_1 PC1_2 PC1_3  : 0.7071067811865476  -0.7071067811865475  0.0  
PC2_1 PC2_2 PC2_3  : 0.7071067811865475  0.7071067811865476  0.0  
PC3_1 PC3_2 PC3_3  : 0.0  0.0  1.0  
CDELT : 0.00014009532001283  0.00014009532001283  1.0  
NAXIS : 0  0


## Plot the reprojected cubes

In [14]:
# Plot the reprojected cube.
%matplotlib widget

fig, ax = plt.subplots(subplot_kw={'projection':rep_cube_0[0,:,:].wcs})
#fig, ax = plt.subplots()

ax.imshow(rep_cube_0.data[0,:,:], origin='lower', cmap='Greys')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# Plot multiple cubes.
%matplotlib widget

#fig, ax = plt.subplots(subplot_kw={'projection':reprojected_cube[0,:,:].wcs})
#fig, ax = plt.subplots()

#ax.imshow(reprojected_cube.data[0,:,:], origin='lower', cmap='Greys')

ax1 = plt.subplot(131, projection=rep_cube_0[0,:,:].wcs)
ax2 = plt.subplot(132, projection=rep_cube_1[0,:,:].wcs)
ax3 = plt.subplot(133, projection=rep_cube_2[0,:,:].wcs)

#ax1 = plt.subplot(111, projection=reprojected_cube[0,:,:].wcs)
plt.subplots_adjust(wspace=0.4)

#m_SP.plot(ax=ax1)
#cropped_map.plot(ax=ax2)

ax1.imshow(rep_cube_0.data[0,:,:], origin='lower', cmap='Greys')
ax2.imshow(rep_cube_1.data[0,:,:], origin='lower', cmap='Greys')
ax3.imshow(rep_cube_2.data[0,:,:], origin='lower', cmap='Greys')

#ax1.yaxis.label.set_visible(False)

#ax1.set_xticks([])
#ax1.set_yticks([])

#ax1.set_ylabel(ylabel=None)


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# Plot all the cubes superposed.
%matplotlib widget

plt.imshow(rep_cube_0.data[0,:,:], origin='lower')
plt.imshow(rep_cube_1.data[0,:,:], origin='lower', alpha=0.5, cmap="Greys")
plt.imshow(rep_cube_2.data[0,:,:], origin='lower', alpha=0.5, cmap="Greys")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
import gc
gc.get_threshold()

(700, 10, 10)

In [18]:
gc.get_count()

(134, 11, 10)

In [22]:
import sys
sys._debugmallocstats?

Signature: sys._debugmallocstats()
Docstring:
Print summary info to stderr about the state of pymalloc's structures.

In Py_DEBUG mode, also perform some expensive internal consistency
checks.
Type:      builtin_function_or_method
